# DataHandling(1)

In [1]:
import numpy as np
import pandas as pd

from sklearn.impute import SimpleImputer, KNNImputer, MissingIndicator

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = {
    'id' : [1000, 1001, 1002, 1003, 1004, 1005, 1006],
    'date' : ['20230101', '20230102', '20230103', '20230104', '20230105', '20230106', '20230107'],
    'age' : [21, 56, 33, 48, 27, 42, 32],
    'income' : [67000, 220000, 97000, 166000, 81000, 157000, 96000],
    'gender' : ['Male', 'Female', 'Female', 'Male', 'Male', 'Female', 'Female'],
    'education' : ['Bachelors', 'PhD', 'Masters', 'Masters', 'Bachelors', 'Bachelors', 'Bachelors'],
    'passed' : [False, True, True, True, False, False, True],
    'measurement' : np.random.randn(7).round(2)
}

df = pd.DataFrame(data)
df

,id,date,age,income,gender,education,passed,measurement
0,1000,20230101,21,67000,Male,Bachelors,False,-0.50
1,1001,20230102,56,220000,Female,PhD,True,0.40
2,1002,20230103,33,97000,Female,Masters,True,-1.52
3,1003,20230104,48,166000,Male,Masters,True,0.04
4,1004,20230105,27,81000,Male,Bachelors,False,-0.75
5,1005,20230106,42,157000,Female,Bachelors,False,-0.34
6,1006,20230107,32,96000,Female,Bachelors,True,1.12


In [3]:
# 컬럼간 상관계수 계산
df.corr()

,id,age,income,passed,measurement
id,1.000000,-0.006285,-0.076124,0.000000,0.377716
age,-0.006285,1.000000,0.984332,0.533450,0.357525
income,-0.076124,0.984332,1.000000,0.413131,0.362715
passed,0.000000,0.533450,0.413131,1.000000,0.340512
measurement,0.377716,0.357525,0.362715,0.340512,1.000000


In [4]:
# 컬럼 정보 확인하기
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           7 non-null      int64  
 1   date         7 non-null      object 
 2   age          7 non-null      int64  
 3   income       7 non-null      int64  
 4   gender       7 non-null      object 
 5   education    7 non-null      object 
 6   passed       7 non-null      bool   
 7   measurement  7 non-null      float64
dtypes: bool(1), float64(1), int64(3), object(3)
memory usage: 527.0+ bytes


In [5]:
# 데이터값 살펴보기
df.values

array([[1000, '20230101', 21, 67000, 'Male', 'Bachelors', False, -0.5],
       [1001, '20230102', 56, 220000, 'Female', 'PhD', True, 0.4],
       [1002, '20230103', 33, 97000, 'Female', 'Masters', True, -1.52],
       [1003, '20230104', 48, 166000, 'Male', 'Masters', True, 0.04],
       [1004, '20230105', 27, 81000, 'Male', 'Bachelors', False, -0.75],
       [1005, '20230106', 42, 157000, 'Female', 'Bachelors', False,
        -0.34],
       [1006, '20230107', 32, 96000, 'Female', 'Bachelors', True, 1.12]],
      dtype=object)

## Missingvalue Processing

In [6]:
df2 = df.copy()
df2.iloc[[3, 5], [1, 2, 4, 5, 6, 7]] = np.nan
df2

,id,date,age,income,gender,education,passed,measurement
0,1000,20230101,21.0,67000,Male,Bachelors,False,-0.50
1,1001,20230102,56.0,220000,Female,PhD,True,0.40
2,1002,20230103,33.0,97000,Female,Masters,True,-1.52
3,1003,NaN,NaN,166000,NaN,NaN,NaN,NaN
4,1004,20230105,27.0,81000,Male,Bachelors,False,-0.75
5,1005,NaN,NaN,157000,NaN,NaN,NaN,NaN
6,1006,20230107,32.0,96000,Female,Bachelors,True,1.12


In [7]:
# 각 컬럼별 결측치 세어보기
df2.isnull().sum()

id             0
date           2
age            2
income         0
gender         2
education      2
passed         2
measurement    2
dtype: int64

In [8]:
# 결측이 포함된 데이터만 보기
df2[df2.isnull().any(axis=1)]

,id,date,age,income,gender,education,passed,measurement
3,1003,NaN,NaN,166000,NaN,NaN,NaN,NaN
5,1005,NaN,NaN,157000,NaN,NaN,NaN,NaN


In [9]:
# 결측치가 포함된 행 삭제
df2.dropna(axis=0)

,id,date,age,income,gender,education,passed,measurement
0,1000,20230101,21.0,67000,Male,Bachelors,False,-0.50
1,1001,20230102,56.0,220000,Female,PhD,True,0.40
2,1002,20230103,33.0,97000,Female,Masters,True,-1.52
4,1004,20230105,27.0,81000,Male,Bachelors,False,-0.75
6,1006,20230107,32.0,96000,Female,Bachelors,True,1.12


In [10]:
# 결측치가 포함된 열 삭제
df2.dropna(axis=1)

,id,income
0,1000,67000
1,1001,220000
2,1002,97000
3,1003,166000
4,1004,81000
5,1005,157000
6,1006,96000


In [11]:
# 결측치를 특정 값으로 치환하기
df2.loc[:, ['age', 'measurement']].fillna(-9999)

,age,measurement
0,21.0,-0.50
1,56.0,0.40
2,33.0,-1.52
3,-9999.0,-9999.00
4,27.0,-0.75
5,-9999.0,-9999.00
6,32.0,1.12


In [12]:
# 결측치를 열 평균값으로 치환하기
df2.loc[:, ['age', 'measurement']].fillna(df2.mean()[['age', 'measurement']])

,age,measurement
0,21.0,-0.50
1,56.0,0.40
2,33.0,-1.52
3,33.8,-0.25
4,27.0,-0.75
5,33.8,-0.25
6,32.0,1.12


In [13]:
# sklearn 패키지를 활용하여 결측치 처리하기
# Imputer를 활용하여 결측치 채워넣기 (특정값)

imp = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=-9999)
imputed = imp.fit_transform(df2.loc[:, ['age', 'measurement']].values)
pd.DataFrame(imputed, columns=['age', 'measurement'])

,age,measurement
0,21.0,-0.50
1,56.0,0.40
2,33.0,-1.52
3,-9999.0,-9999.00
4,27.0,-0.75
5,-9999.0,-9999.00
6,32.0,1.12


In [14]:
# Imputer를 활용하여 결측치 채워넣기 (평균)
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imputed = imp.fit_transform(df2.loc[:, ['age', 'measurement']].values)
pd.DataFrame(imputed, columns=['age', 'measurement'])

,age,measurement
0,21.0,-0.50
1,56.0,0.40
2,33.0,-1.52
3,33.8,-0.25
4,27.0,-0.75
5,33.8,-0.25
6,32.0,1.12


In [15]:
# Imputer를 활용하여 결측치 채워넣기 (n_neighbor)
knn_imp = KNNImputer(n_neighbors=2, weights='uniform')
imputed = knn_imp.fit_transform(df2.loc[:, ['age', 'measurement']].values)
pd.DataFrame(imputed, columns=['age', 'measurement'])

,age,measurement
0,21.0,-0.50
1,56.0,0.40
2,33.0,-1.52
3,33.8,-0.25
4,27.0,-0.75
5,33.8,-0.25
6,32.0,1.12


In [16]:
# 문자열에 대한 결측치 처리하기
# 특정값으로 치환하기
df2.loc[:, ['gender', 'education']].fillna('Unknown')

,gender,education
0,Male,Bachelors
1,Female,PhD
2,Female,Masters
3,Unknown,Unknown
4,Male,Bachelors
5,Unknown,Unknown
6,Female,Bachelors


In [17]:
# mode(최빈값)을 활용하여 치환하기
df2.loc[:, ['gender', 'education']].fillna(df2.mode().iloc[0])

,gender,education
0,Male,Bachelors
1,Female,PhD
2,Female,Masters
3,Female,Bachelors
4,Male,Bachelors
5,Female,Bachelors
6,Female,Bachelors


In [18]:
# sklearn을 활용하여 문자열 결측치를 처리하기
imp = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value='Unknown')
imputed = imp.fit_transform(df2.loc[:, ['gender', 'education']].values)
pd.DataFrame(imputed, columns=['gender', 'education'])

,gender,education
0,Male,Bachelors
1,Female,PhD
2,Female,Masters
3,Unknown,Unknown
4,Male,Bachelors
5,Unknown,Unknown
6,Female,Bachelors


In [19]:
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputed = imp.fit_transform(df2.loc[:, ['gender', 'education']].values)
pd.DataFrame(imputed, columns=['gender', 'education'])

,gender,education
0,Male,Bachelors
1,Female,PhD
2,Female,Masters
3,Female,Bachelors
4,Male,Bachelors
5,Female,Bachelors
6,Female,Bachelors


In [20]:
# 문자열 데이터를 날짜 데이터로 처리하기
df2.date

0    20230101
1    20230102
2    20230103
3         NaN
4    20230105
5         NaN
6    20230107
Name: date, dtype: object

In [21]:
df2.date.dtype

dtype('O')

In [22]:
# to_datetime 함수를 활용하여 str -> datetime으로 변환
df2.date = pd.to_datetime(df2.date)
df2.date.dtype

dtype('<M8[ns]')

In [23]:
df2.date[1].dayofweek

0

In [24]:
df2.date[1].weekofyear

1

In [25]:
df2['date'] = pd.date_range(start=df2.date.min(), end=df2.date.max(), freq='D')
df2['dayofweek'] = df2['date'].apply(lambda x: x.dayofweek)
df2

,id,date,age,income,gender,education,passed,measurement,dayofweek
0,1000,2023-01-01,21.0,67000,Male,Bachelors,False,-0.50,6
1,1001,2023-01-02,56.0,220000,Female,PhD,True,0.40,0
2,1002,2023-01-03,33.0,97000,Female,Masters,True,-1.52,1
3,1003,2023-01-04,NaN,166000,NaN,NaN,NaN,NaN,2
4,1004,2023-01-05,27.0,81000,Male,Bachelors,False,-0.75,3
5,1005,2023-01-06,NaN,157000,NaN,NaN,NaN,NaN,4
6,1006,2023-01-07,32.0,96000,Female,Bachelors,True,1.12,5


# DataHandling(2)

In [28]:
path = '/content/drive/MyDrive/Colab Notebooks/fastcampus/data/'

sales_df = pd.read_csv(path+'sales_data.csv')
social1_df = pd.read_csv(path+'social1.csv')
social2_df = pd.read_csv(path+'social2.csv')
social3_df = pd.read_csv(path+'social3.csv')

## merge

In [30]:
# 컬럼명 변경하기
sales_df.columns = ['date', 'sum']
sales_df.head()

,date,sum
0,2017-01-01,4602.0
1,2017-01-02,6680.0
2,2017-01-03,4125.0
3,2017-01-04,3743.0
4,2017-01-05,3536.0


In [31]:
social1_df.columns = ['date', 'ratio1']
social1_df.head()

,date,ratio1
0,2017-01-02,4.917119
1,2017-01-09,15.147494
2,2017-01-16,3.574863
3,2017-01-23,0.000000
4,2017-01-30,3.555632


In [32]:
social2_df.columns = ['date', 'key1', 'key2', 'key3', 'key4']
social2_df

,date,key1,key2,key3,key4
0,2018-01-08,18,22,2,1
1,2018-01-15,14,20,1,1
2,2018-01-22,16,16,2,1
3,2018-01-29,18,19,3,1
4,2018-02-05,15,20,2,1
...,...,...,...,...,...
99,2019-12-02,24,20,2,1
100,2019-12-09,27,26,1,1
101,2019-12-16,34,20,2,1
102,2019-12-23,40,22,2,1


In [33]:
social3_df.columns = ['date', 'ratio2']
social3_df

,date,ratio2
0,2015-12-28,14.20355
1,2016-01-04,16.75552
2,2016-01-11,17.78589
3,2016-01-18,18.10339
4,2016-01-25,19.28353
...,...,...
209,2019-12-30,17.81884
210,2020-01-06,21.73066
211,2020-01-13,22.23686
212,2020-01-20,19.26256


In [34]:
# Inner Merge(Join) 을 통해 데이터 합치기
result_df = sales_df.merge(social1_df, on='date', how='inner')
result_df

,date,sum,ratio1
0,2017-01-02,6680.0,4.917119
1,2017-01-09,2879.0,15.147494
2,2017-01-16,2726.0,3.574863
3,2017-01-23,2641.0,0.000000
4,2017-01-30,3171.0,3.555632
...,...,...,...
150,2019-12-02,8059.0,35.471713
151,2019-12-09,6325.0,39.154263
152,2019-12-16,4407.0,36.246683
153,2019-12-23,4045.0,34.252529


In [35]:
result_df = result_df.merge(social2_df, on='date', how='inner')
result_df

,date,sum,ratio1,key1,key2,key3,key4
0,2018-01-08,3611.0,11.143802,18,22,2,1
1,2018-01-15,3081.0,13.436022,14,20,1,1
2,2018-01-22,1920.0,14.085997,16,16,2,1
3,2018-01-29,2716.0,16.685897,18,19,3,1
4,2018-02-05,2162.0,15.316719,15,20,2,1
...,...,...,...,...,...,...,...
97,2019-12-02,8059.0,35.471713,24,20,2,1
98,2019-12-09,6325.0,39.154263,27,26,1,1
99,2019-12-16,4407.0,36.246683,34,20,2,1
100,2019-12-23,4045.0,34.252529,40,22,2,1


In [36]:
result_df = result_df.merge(social3_df, on='date', how='inner')
result_df

,date,sum,ratio1,key1,key2,key3,key4,ratio2
0,2018-01-08,3611.0,11.143802,18,22,2,1,17.18684
1,2018-01-15,3081.0,13.436022,14,20,1,1,19.32546
2,2018-01-22,1920.0,14.085997,16,16,2,1,19.24459
3,2018-01-29,2716.0,16.685897,18,19,3,1,20.31689
4,2018-02-05,2162.0,15.316719,15,20,2,1,19.92152
...,...,...,...,...,...,...,...,...
97,2019-12-02,8059.0,35.471713,24,20,2,1,18.29209
98,2019-12-09,6325.0,39.154263,27,26,1,1,18.36997
99,2019-12-16,4407.0,36.246683,34,20,2,1,19.24159
100,2019-12-23,4045.0,34.252529,40,22,2,1,16.76451


In [37]:
# 1줄로 처리 가능, 가독성 때문에 비추천
test_df = sales_df.merge(social1_df, on='date', how='inner').merge(social2_df, on='date', how='inner').merge(social3_df, on='date', how='inner')
test_df

,date,sum,ratio1,key1,key2,key3,key4,ratio2
0,2018-01-08,3611.0,11.143802,18,22,2,1,17.18684
1,2018-01-15,3081.0,13.436022,14,20,1,1,19.32546
2,2018-01-22,1920.0,14.085997,16,16,2,1,19.24459
3,2018-01-29,2716.0,16.685897,18,19,3,1,20.31689
4,2018-02-05,2162.0,15.316719,15,20,2,1,19.92152
...,...,...,...,...,...,...,...,...
97,2019-12-02,8059.0,35.471713,24,20,2,1,18.29209
98,2019-12-09,6325.0,39.154263,27,26,1,1,18.36997
99,2019-12-16,4407.0,36.246683,34,20,2,1,19.24159
100,2019-12-23,4045.0,34.252529,40,22,2,1,16.76451


## iloc

In [39]:
# iloc을 이용하여 2번째 컬럼부터 마지막 컬럼까지의 데이터만 불러오기
test_df.iloc[:, 1:]

,sum,ratio1,key1,key2,key3,key4,ratio2
0,3611.0,11.143802,18,22,2,1,17.18684
1,3081.0,13.436022,14,20,1,1,19.32546
2,1920.0,14.085997,16,16,2,1,19.24459
3,2716.0,16.685897,18,19,3,1,20.31689
4,2162.0,15.316719,15,20,2,1,19.92152
...,...,...,...,...,...,...,...
97,8059.0,35.471713,24,20,2,1,18.29209
98,6325.0,39.154263,27,26,1,1,18.36997
99,4407.0,36.246683,34,20,2,1,19.24159
100,4045.0,34.252529,40,22,2,1,16.76451


In [40]:
# iloc을 이용하여 2번째 컬럼부터 마지막 이전 컬럼까지의 데이터만 불러오기
test_df.iloc[:, 1:-1]

,sum,ratio1,key1,key2,key3,key4
0,3611.0,11.143802,18,22,2,1
1,3081.0,13.436022,14,20,1,1
2,1920.0,14.085997,16,16,2,1
3,2716.0,16.685897,18,19,3,1
4,2162.0,15.316719,15,20,2,1
...,...,...,...,...,...,...
97,8059.0,35.471713,24,20,2,1
98,6325.0,39.154263,27,26,1,1
99,4407.0,36.246683,34,20,2,1
100,4045.0,34.252529,40,22,2,1


In [41]:
# iloc을 이용하여 3번째, 마지막 이전 컬럼 데이터만 불러오기
test_df.iloc[:, [2,-1]]

,ratio1,ratio2
0,11.143802,17.18684
1,13.436022,19.32546
2,14.085997,19.24459
3,16.685897,20.31689
4,15.316719,19.92152
...,...,...
97,35.471713,18.29209
98,39.154263,18.36997
99,36.246683,19.24159
100,34.252529,16.76451


In [43]:
# iloc을 이용하여 2번째부터 10번째 행까지의 데이터만 불러오기
test_df.iloc[1:10, :]

,date,sum,ratio1,key1,key2,key3,key4,ratio2
1,2018-01-15,3081.0,13.436022,14,20,1,1,19.32546
2,2018-01-22,1920.0,14.085997,16,16,2,1,19.24459
3,2018-01-29,2716.0,16.685897,18,19,3,1,20.31689
4,2018-02-05,2162.0,15.316719,15,20,2,1,19.92152
5,2018-02-12,1532.0,8.365063,15,22,3,1,19.70886
6,2018-02-19,666.0,17.441637,22,22,3,1,26.36134
7,2018-02-26,217.0,21.114573,32,20,4,1,32.32193
8,2018-03-05,53.0,21.860698,31,23,5,1,34.81698
9,2018-03-12,77.0,23.945233,34,27,7,1,52.61187


In [44]:
# 데이터 나눠보기
len(test_df)

102

In [45]:
test_df.iloc[0:int(len(test_df) * 0.6)]

,date,sum,ratio1,key1,key2,key3,key4,ratio2
0,2018-01-08,3611.0,11.143802,18,22,2,1,17.18684
1,2018-01-15,3081.0,13.436022,14,20,1,1,19.32546
2,2018-01-22,1920.0,14.085997,16,16,2,1,19.24459
3,2018-01-29,2716.0,16.685897,18,19,3,1,20.31689
4,2018-02-05,2162.0,15.316719,15,20,2,1,19.92152
...,...,...,...,...,...,...,...,...
56,2019-02-18,915.0,47.653936,28,21,2,1,27.88594
57,2019-02-25,439.0,49.369255,35,20,3,1,34.94878
58,2019-03-04,150.0,51.071113,30,17,5,1,39.39974
59,2019-03-11,47.0,57.349717,32,22,3,2,38.58803


In [46]:
test_df.iloc[int(len(test_df) * 0.6):]

,date,sum,ratio1,key1,key2,key3,key4,ratio2
61,2019-03-25,8.0,68.485827,63,20,6,1,47.01072
62,2019-04-01,-7.0,74.202915,92,21,6,1,46.65129
63,2019-04-08,10.0,76.352833,67,18,8,2,52.25543
64,2019-04-15,4.0,86.919734,69,18,9,2,69.36739
65,2019-04-22,13.0,93.377178,72,16,14,1,88.39633
66,2019-04-29,567.0,96.492443,70,16,12,2,80.47684
67,2019-05-06,340.0,96.867428,63,22,18,2,90.06769
68,2019-05-13,513.0,98.155840,69,21,20,2,100.00000
69,2019-05-20,316.0,100.000000,73,17,21,2,83.19355
70,2019-05-27,372.0,93.561786,85,23,17,1,71.94033


In [47]:
# iloc을 이용하여 행, 열 선택하여 데이터 불러오기
test_df.iloc[11:20, 2:5]

,ratio1,key1,key2
11,36.562055,54,30
12,42.752202,62,26
13,44.746356,57,24
14,53.796008,52,29
15,56.051690,63,26
16,65.299412,46,25
17,65.326334,50,19
18,73.972155,43,20
19,75.529787,48,21


## loc filtering

In [49]:
# loc을 이용하여 특정 컬럼의 조건으로 데이터 필터링 하기
test_df.loc[(test_df['ratio1'] > 40.0) & (test_df['ratio2'] > 30.0)]

,date,sum,ratio1,key1,key2,key3,key4,ratio2
12,2018-04-02,-55.0,42.752202,62,26,7,1,75.37291
13,2018-04-09,-11.0,44.746356,57,24,8,2,64.68579
14,2018-04-16,33.0,53.796008,52,29,9,1,78.09261
15,2018-04-23,-12.0,56.051690,63,26,12,1,72.39261
16,2018-04-30,12.0,65.299412,46,25,12,1,83.96932
17,2018-05-07,-4.0,65.326334,50,19,13,1,76.50212
18,2018-05-14,2.0,73.972155,43,20,14,0,83.05277
19,2018-05-21,320.0,75.529787,48,21,10,1,71.53597
20,2018-05-28,96.0,78.827737,46,26,12,1,81.98047
21,2018-06-04,348.0,75.816315,54,22,12,1,76.60995
